In [ ]:
#default_exp gradients.estimate

In [ ]:
#     def get_dF_js_idM(self, M, N, M_tangent_bundle_sub, N_tangent_bundle, selectedpoints, dim = None):

#         if dim == None:
#             dim = self.dim
#         m = self.m
#         affinity_matrix = M.geom.affinity_matrix

#         nsel = len(selectedpoints)
#         dF = np.zeros((nsel, m, dim))

#         for i in range(nsel):
#             pt = selectedpoints[i]
#             neighborspt = affinity_matrix[selectedpoints[i]].indices
#             deltap0 = M.data[neighborspt, :] - M.data[pt, :]
#             deltaq0 = N.data[neighborspt, :] - N.data[pt, :]
#             projected_M = np.matmul(M_tangent_bundle_sub.tangent_bases[i, :, :].transpose(),
#                                     deltap0.transpose()).transpose()
#             tan_proj = np.dot(N_tangent_bundle.tangent_bases[selectedpoints[i]],N_tangent_bundle.tangent_bases[selectedpoints[i]].transpose())
#             projected_N = np.dot(deltaq0, tan_proj)

#             lr = LinearRegression()
#             weights = affinity_matrix[selectedpoints[i]].data
#             lr.fit(projected_M, projected_N, weights)
#             dF[i, :, :][:, :] = lr.coef_#np.linalg.lstsq(projected_M, deltaq0)[0]#np.matmul(a, b).transpose()
#         return (dF)

In [ ]:
def get_grads():

def get_grads_wls(data, 
                       embedding,
                       geom,
                       tangent_bases_M,
                       tangent_bases_N,
                       selected_points):

In [ ]:
#export

import numpy as np
from sklearn.linear_model import LinearRegression

def get_grads_pullback(data, 
               embedding,
               geom,
               tangent_bases_M,
               tangent_bases_N,
               selected_points):
    
    affinity_matrix = geom.affinity_matrix
    m = embedding.shape[1]
    d = tangent_bases_M.shape[2]
    nsel = selected_points.shape[0]
    dF = np.zeros((nsel, d, m))
    
    
    for i in range(nsel):
        pt = selected_points[i]
        neighborspt = affinity_matrix[pt].indices
        deltap0 = data[neighborspt, :] - data[pt, :]
        deltaq0 = embedding[neighborspt, :] - embedding[pt, :]
        projected_M = np.einsum('b d, i b -> i d', tangent_bases_M[i], deltap0)
        tangent_bases_N_outer = np.einsum('m d,  n d ->  m n', tangent_bases_N[i], tangent_bases_N[i])
        projected_N = np.einsum('m n, i n -> i m', tangent_bases_N_outer, deltaq0)
        lr = LinearRegression()
        lr.fit(projected_M, projected_N)
        #weights = affinity_matrix[selectedpoints[i]].data
        #lr.fit(projected_M, projected_N, weights)
        dF[i] = lr.coef_.transpose()#np.linalg.lstsq(projected_M, deltaq0)[0]

    return(dF)